In [3]:
import msprime
import numpy as np
import pandas as pd
import argparse
import gzip
import csv

def create_rate_map(map_file):
    # Initialize lists to store positions and rates
    positions = [0]
    gmaps = [0]

    # Read positions from map.txt file
    with open(map_file, 'r') as f:
        for line in f:
            if line.strip():  # Check if line is not empty
                parts = line.split()
                position = float(parts[3])  # Extract position (assuming it's the third column)
                positions.append(position)
                gmap = float(parts[2])
                gmaps.append(gmap)

    # Calculate combination rates and record them
    combined_rates = []
    for i in range(1, len(positions)):
        pos1 = positions[i - 1]
        pos2 = positions[i]
        gmap1 = gmaps[i - 1]
        gmap2 = gmaps[i]
        combination_rate = ((gmap2 - gmap1) / (pos2 - pos1)) * 1e-6
        combined_rates.append(combination_rate)
    
    rate_map = msprime.RateMap(
        position=positions,
        rate=combined_rates
    )

    with open("recombination_rates_chr1.csv", "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
    
        # Write header
        writer.writerow(["Position", "Rate"])
    
        # Write data rows
        for i in range(len(positions)-1):
            writer.writerow([positions[i], combined_rates[i]])
    
    return rate_map


def export_vcf(mts, name):
    # Output VCF (Variant Call Format) file
    vcf_file = name + ".vcf"
    with open(vcf_file, "w") as vcf_out:
        mts.write_vcf(vcf_out)


def sim_one_const(pop_size, sample_size, rate_map, mu_rate):
    # Create a demographic model for a constant-size population
    demographic_model = msprime.Demography()
    demographic_model.add_population(initial_size=pop_size)
    
    # Simulate the tree sequence
    ts = msprime.sim_ancestry(samples=sample_size, demography=demographic_model, recombination_rate=rate_map,
                              ploidy=2)
    
    # Simulate mutations
    mts = msprime.sim_mutations(ts, rate= mu_rate)
    
    return mts


In [4]:
# Example usage in an IPython notebook
if __name__ == '__main__':
    pop_size = 20000
    sample_size = 1000
    mu_rate = 1e-8  # Mutation rate per base pair per generation
    map_file = 'plink.chr1.GRCh38.map'
    output_prefix = 'simulation_output'

    # Create rate map from genetic map file
    rate_map = create_rate_map(map_file)
    
    # Run the simulation
    mts = sim_one_const(pop_size, sample_size, rate_map, mu_rate)
    
    # Export VCF
    export_vcf(mts, output_prefix)

The provenance information for the resulting tree sequence is 8.64MB. This is nothing to worry about as provenance is a good thing to have, but if you want to save this memory/storage space you can disable provenance recording by setting record_provenance=False
